### Deeper Network Analysis 

Now let us extract information from the popular cryptocurrency subreddits to find communities of influencial authors. We offer a list of top subreddits in the field, and then build a graph recursively. 

Code adapted from: https://medium.com/social-media-theories-ethics-and-analytics/network-analysis-from-social-media-data-with-networkx-13605d711590

In [1]:
import praw 
import prawcore
import pandas as pd
import datetime as dt
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np
import requests
import json
import sys
import traceback
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
client_id = "rax4hz8_P0_ttyt8crPYEg"
client_secret = "7zKse44yWl7Dge7kYQV6fKs20IzLdw"
user_agent = "data_analysis_crypto_sentiment v1.0 by /u/K_C_7"

reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

In [3]:
popular_crypto_subs = ('bitcoin+'\
                       'btc+'\
                       'CryptoMarkets+'\
                       'bitcoinbeginners+'\
                       'CryptoCurrencies+'\
                       'altcoin+'\
                       'icocrypto+'\
                       'CryptoCurrencyTrading+'\
                       'Crypto_General+'\
                       'ico+'\
                       'blockchain+'\
                       'Best Altcoin Subreddits+'\
                       'ethereum+'\
                       'Ripple+'\
                       'litecoin+'\
                       'Monero+'\
                       'Stellar+'\
                       'binance+'\
                       'CoinBase+'\
                       'ledgerwallet+'\
                       'defi+'\
                       'EthTrader+'\
                       'ethfinance'\
                       'LitecoinTraders')

We now define a recursive function ```recursive_node_adder``` whose inputs are the graph in its current state, a comment and a comment's parent author. The goal is to construct a graph $g$ whose nodes are authors of comments and submissions, and the edges which connect them signify that an author has repsonded to another author's reddit input. In this way, it is sensible to define a <b> directed graph </b>, where now edges have a direction, and if redditor $r$ responds to a comment/submission of redditor $r'$, then $r \to r'$ is a directed link between $r$ and $r'$.

In [6]:
def recursive_node_adder(g, comment, parent_author):
    '''Recursively process comments and add them to the graph g'''    
    if not (comment.author is None):
        # Check if we have the node already in our graph
        if (comment.author not in g.nodes):
            g.add_node(comment.author)
        # Create an edge between this comment author and the parent author
        g.add_edge(comment.author, parent_author)
        # Iterate through the comments
        for reply in comment.replies.list():
            if isinstance(reply, praw.models.MoreComments):
                continue
            if (comment.author is None) or (parent_author is None):
                continue
            # Recursively process this reply
            recursive_node_adder(g, reply, comment.author)

Now we extract the submissions and their metadata from the popular cryptocurrency subreddits over the past month. For the sake of time, we limit the search to the top $n$ submissions where $n$ is given the name ``` limit ``` below. If $n$ is too large during the processing one might interrupt the loop after a large enough number of iterations. 

In [ ]:
# Create a DIRECTED graph 
g = nx.DiGraph()
subreddit = popular_crypto_subs
breadthCommentCount = 10

targetSub = reddit.subreddit(subreddit)
submissions = targetSub.top(limit=1000, time_filter='month') #may stop after a significant num of its

i=0

for post in submissions:
    if i%50 == 0:
        print(i)
    #print (post.author, "-", post.title)
    if not (post.author == None):
        # Check if we have the node already in our graph
        if (post.author not in g.nodes):
            g.add_node(post.author)
        post.comment_limit = breadthCommentCount
        # Get the top few comments
        for comment in post.comments.list():
            # Skip MoreComment objects, which don't have authors
            if isinstance(comment, praw.models.MoreComments):
                continue
            # Recursively process this reply
            recursive_node_adder(g, comment, post.author)
    i+=1

In [23]:
labels = list(g.nodes())
#convert Subreddit instances into their display names for convenience
nodes_new = list(g.nodes()) #initialise
for index, node in enumerate(labels):
    if isinstance(node, str) == False:
        nodes_new[index] = node.name
        
mapping = dict(zip(g.nodes(), nodes_new))        
g = nx.relabel_nodes(g, mapping)

#json storage... could be useful?
from networkx.readwrite import json_graph 
data = json_graph.node_link_data(g)
H = json_graph.node_link_graph(data, directed=True)

In [39]:
#save the graph
nx.write_gml(g, "/Users/kc/Documents/StatCompVis/CryptoProject/Optimal-Cryptocurrency-Trading-Strategies-Step2/Reddit_Analysis/graph_for_backtesting.gml")

In [41]:
#export node lists
nodes_list = list(g.nodes())
with open('g_nodes.txt', 'w') as filehandle:
    for node in nodes_list:
        filehandle.write('%s\n' % node)

Okay, so now we can look at the graph and its properties (might be easier in R for plotting/interactivity etc purposes).

In [13]:
nx.info(g)

'DiGraph with 2965 nodes and 5395 edges'